In [1]:
from collections import deque 
from collections import Counter
import os                           
import csv
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def extract_prices(text):  #פונקציה שמשאירה רק את המחיר
    prices = []
    pattern = r'\$\d+(?:\,\d+)?'
    matches = re.findall(pattern, text)
    for match in matches:
        prices.append(match)
    return prices

def find_example_line(text,toSearch): #פונקציה למשיכת הנתונים מהדף של הדירה
    missing = "None"
    lines = text.split('\n')
    for line in lines:
        if line.startswith(toSearch):
            return line
    return missing

def find_property_links(input_string):  #פונקציה שמטפלת בכל הלינקים שקיבלנו שיהיו בצורה שאנחנו צריכים
        links = []
        for match in re.findall(r'<a href="/property/[^/][^"]*"', input_string):
            links.append(match.split('"')[1])
        return links
def extract_bedrooms(string): #פונקציה שמשאירה רק את כמות חדרי האמבטיות
    test = ""
    if string[3] == 'e':
        test = string[0]
        return test
    else:
        test = "None"
        return test
def extract_bathrooms(string): #פונקציה שמשאירה רק את כמות חדר השינה
    test = ""
    if string[10] == 'a':
        test = string[7]
        return test
    else:
        test = "None"
        return test
def extract_Sqf(string): #פונקציה שמשאירה רק את המטר מרובע במספר
    if string[21] == 'S':
        sqft = re.findall(r"\b[\d,]+\b\s*Sq Ft", string)[0]
        sqft = re.sub(",", "", re.findall(r"\d{1,4}(?:,\d{3})*", sqft)[0])
        return sqft
    test = "None"
    return test

In [3]:
import undetected_chromedriver as uc #פתיחת העמוד הראשי והפעלת מונע חסימות
driver = uc.Chrome() 
driver.get("https://www.homes.com/las-vegas-nv/sold/") 

In [4]:
links_list = []
refined_info = []
k = 1                    #ריצה על כל העמודים 
while(k <= 28):
    driver.get("https://www.homes.com/las-vegas-nv/"+"sold/"+"p"+str(k)+"/")
    myDiv = driver.find_elements(By.CLASS_NAME, 'price-container')
    links = driver.find_elements(By.CSS_SELECTOR, "#placardContainer > div.search-placards-container > ul > li")
    i = 0
    size = len(links)
    while( i < size):       #הכנסת כל HTML של העמודים
        links_list += links[i].get_attribute('outerHTML')
        i += 1
    k+=1

In [5]:
string = ''.join(links_list)   #ניקוי כל הדברים שלא צריך שהוצמדו ללינקים
links_list = find_property_links(string)
links_list = [elem.split(',') for elem in links_list]

In [6]:
new_list = []      #הכנסה של הלינקים למערך חדש אחרי שנוקו
for item in links_list:
    if item not in new_list:
        new_list.append(item)

In [7]:
df = pd.DataFrame(new_list) #שמירת הלינקים בדאטה פריים
df.to_csv("try5.csv", index = False)

In [8]:
with open('try5.csv', 'r') as csvfile: #טעינה של הלינקים מדאטה פריים והוצאה של הערך 0 שנשבחר תמיד אחרי שפותחים מדאטה פריים
    csvreader = csv.reader(csvfile)
    new_list = list(csvreader)
    
new_list.pop(0)

['0']

In [9]:
YBD = "Year Built Details" #19
R = "Roof" #5
PST = "Property Sub Type" #18
AG = "Attached Garage" #16
S = "Sewer" #6
AC = "Attribution Contact" #20
PB = "Possible Bedrooms" #18
Fire = "Fireplace " #10
C = "Cooling" #8
AF = "Association Fee" #16
Floor = "Flooring" #9
WF = "Window Features" #16
PF = "Parking Features" #17
PAPF = "Patio And Porch Features" #25
H = "Heating" #8
Fencing = "Fencing" #8
YBD_list = []
R_list = []
PST_list = []
AG_list = []
S_list = []
AC_list = []
PB_list = []
Fire_list = []
C_list = []                                       #הגדרה של כל מה שאנחנו מחפשים ובאיזה אורך כל מילה
AF_list = []
Floor_list = []
WF_list = []
PF_list = []
PAPF_list = []
H_list = []
Fencing_list = []
Price_list = []
beds_list = []
Baths_list = []
Sq_Ft_list = []
Adress_list = []

In [13]:
size_list = len(new_list) #בדיקה של כמות הלינקים על מנת שנדע מתי לעצור את הלולאה
p = 0
while(p < size_list):
    string = str(new_list[p]) #השמטה של מה שלא צריך בטעינה של הלינקים והוספה לכתובת הראשית ובעזרת הלולאה נעבור דירה דירה וניקח מה שצריך
    string = string[2:-2]
    driver.get("https://www.homes.com"+string)
    string2 = ""
    elems = driver.find_elements(By.CLASS_NAME, 'property-info-features')
    for elem in elems:
        string2 += elem.text
        
    price_elem = driver.find_element(By.ID, 'price')    #5 אלמנטים שניקח בחלק העליון של הדף שנמצאים בקלאס אחר מהשאר
    price_elem_text = price_elem.text    
    Price_list.append(price_elem_text)
    
    address_ele = driver.find_element(By.TAG_NAME, 'h1')
    address_ele_text = address_ele.text
    Adress_list.append(address_ele_text)
    beds_list.append(extract_bedrooms(string2))
    Baths_list.append(extract_bathrooms(string2))
    Sq_Ft_list.append(extract_Sqf(string2))
    
    
    Text = driver.find_element(By.ID, 'amenities-container')
    Text = Text.text
    YBD_list.append(find_example_line(Text,YBD)[20:])  
    R_list.append(find_example_line(Text,R)[6:])                 #אחרי שנמצא אם השם שחיםשנו ניקח רק את מה שאחריו ולא את השם עצמו שיופיע בשם העמודה
    PST_list.append(find_example_line(Text,PST)[19:])  
    AG_list.append(find_example_line(Text,AG)[17:])  
    S_list.append(find_example_line(Text,S)[7:])  
    AC_list.append(find_example_line(Text,AC)[21:]) 
    PB_list.append(find_example_line(Text,PB)[19:]) 
    Fire_list.append(find_example_line(Text,Fire)[11:]) 
    C_list.append(find_example_line(Text,C)[9:]) 
    AF_list.append(find_example_line(Text,AF)[17:]) 
    Floor_list.append(find_example_line(Text,Floor)[10:]) 
    WF_list.append(find_example_line(Text,WF)[17:]) 
    PF_list.append(find_example_line(Text,PF)[18:]) 
    PAPF_list.append(find_example_line(Text,PAPF)[26:]) 
    H_list.append(find_example_line(Text,H)[9:]) 
    Fencing_list.append(find_example_line(Text,Fencing)[9:]) 
    p += 1

In [14]:
df = pd.DataFrame(zip(Price_list,beds_list, Baths_list, Sq_Ft_list,Adress_list,YBD_list,R_list,PST_list,AG_list,S_list,AC_list,PB_list,Fire_list,C_list,AF_list,Floor_list,WF_list,PF_list,PAPF_list,H_list,Fencing_list), columns=['Price', 'Beds_Count', 'Baths_Count', 'Sq_Ft_Count', 'Address','Year Built Details','Roof','Property Sub Type','Attached Garage','Sewer','Attribution Contact','Possible Bedrooms','Fireplace','Cooling','Association Fee','Flooring','Window Features','Parking Features','Patio And Porch Features','Heating','Fencing'])
df.to_csv('table3.csv')    #שמירה של כל מה שלקחנו בדאטה סט וכל ערך יכנס בדיוק לעמודה שלו ולפי הסדר ששמרנו בליסטים